In [ ]:
# Install your library here, for example the fynesse template
# is set up to be pip installable
%pip install git+https://github.com/jeffrey-22/ads.git
import os, fynesse
# 2 min

In [1]:
# Import local fynesse module. Do NOT run this cell if the notebook is not run from the repo - this is a quick hack for local runs
%load_ext autoreload
%autoreload 2
import os, sys, IPython
from pathlib import Path
try:
    notebook_path = Path(IPython.get_ipython().run_line_magic('pwd', '')).as_posix()
except AttributeError:
    notebook_path = Path(__file__).resolve().as_posix()
script_path = os.path.abspath(notebook_path)
project_path = os.path.abspath(os.path.join(script_path, '..'))
sys.path.append(project_path)
import fynesse

In [2]:
# Create connections. The connections will be reused throughout the modules
pp_database_details = fynesse.access.retreive_database_details()
# fynesse.access.create_database(pp_database_details)
pp_database_conn = fynesse.access.create_connection(pp_database_details)

In [3]:
import pandas as pd
query = "SELECT * FROM pp_data LIMIT 1000;"
df = pd.read_sql_query(query, pp_database_conn)

C:\Users\86189\AppData\Local\Temp\ipykernel_20932\783174548.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, pp_database_conn)


In [6]:
df.iloc[0:20]

,transaction_unique_identifier,price,date_of_transfer,postcode,property_type,new_build_flag,tenure_type,primary_addressable_object_name,secondary_addressable_object_name,street,locality,town_city,district,county,ppd_category_type,record_status,db_id
0,{CB3AC8B2-C262-4BEF-B21E-6C1DD930B692},125000,1998-05-15,LS25 5PD,T,Y,F,3,,STEETON WAY,SOUTH MILFORD,LEEDS,SELBY,NORTH YORKSHIRE,A,A,1
1,{61EFBFC1-EDF3-49EE-AE92-6C1DDD2C887F},26500,1998-08-12,DL15 8HZ,S,N,F,2,,HIGHFIELD GARDENS,HOWDEN LE WEAR,CROOK,WEAR VALLEY,DURHAM,A,A,2
2,{BD67F778-697A-446C-9F93-64F609F3C01D},147000,1998-08-18,SK9 5QW,T,N,F,2,,GATCOMBE MEWS,WILMSLOW,WILMSLOW,MACCLESFIELD,CHESHIRE,A,A,3
3,{9B0871A9-5877-46C7-AFA3-64F6146E5E00},61950,1998-10-30,SO53 4HA,D,Y,F,3,,WHEAT CLOSE,CHANDLER'S FORD,EASTLEIGH,TEST VALLEY,HAMPSHIRE,A,A,4
4,{38D592BE-FC4D-46EB-8024-64F6148D6803},129000,1998-07-15,WR14 1QQ,S,N,F,16,,HORNYOLD ROAD,MALVERN,MALVERN,MALVERN HILLS,WORCESTERSHIRE,A,A,5
5,{B2A2113F-EC7E-413B-99D8-617249AE402A},58250,1998-12-15,CH2 4RD,D,Y,F,41,,DOVE CLOSE,ELTON,CHESTER,CHESTER,CHESHIRE,A,A,6
6,{6865805B-93CD-4841-AF88-617259B204EF},35000,1998-08-14,NR30 5HL,S,N,F,3,,EASTERN AVENUE,CAISTER ON SEA,GREAT YARMOUTH,GREAT YARMOUTH,NORFOLK,A,A,7
7,{25B485DC-DB5F-4055-BC7C-617261431796},40500,1998-06-24,BN1 6DF,F,N,L,"KARENZA COURT, 69 - 71",FLAT 8,SPRINGFIELD ROAD,BRIGHTON,BRIGHTON,BRIGHTON AND HOVE,BRIGHTON AND HOVE,A,A,8
8,{569AF792-7D13-4EE7-8921-61726920EA9E},53400,1998-12-10,PR5 4LW,T,Y,L,30,,DEVONPORT CLOSE,WALTON LE DALE,PRESTON,SOUTH RIBBLE,LANCASHIRE,A,A,9
9,{42D1E987-6581-4E48-B339-61726DBAA156},156000,1998-10-30,N1 9RW,F,Y,L,"ICE WHARF, 17",FLAT 335,NEW WHARF ROAD,LONDON,LONDON,ISLINGTON,GREATER LONDON,A,A,10


The schema stores price transaction data about the traded households.
- ```transaction_unique_identifier``` and ```db_id``` are indices for transactions and for our database respectively.
- ```price``` describes, in GBP, the price of the household.
- ```postcode```, ```primary_addressable_object_name```, ```secondary_addressable_object_name```, ```street```, ```locality```, ```town_city```, ```district```, ```county``` describe the address. We will ultimately only use the postcode to find the latitude and longitude.
- ```property_type``` describes one of: Detached, Semi-detached, Terraced, Flat/maisonette, Other, indicated by the initials.
- ```new_build_flag```, ```tenure_type```, ```ppd_category_type```, ```record_status``` likely describe some categories, but they are not very helpful as we are ultimately not given these when predicting.

We will do some sanity checks and general visualisation of the data in the address part.

In [ ]:
# Write the code you need for creating the table, downloading and uploading the data here. You can use as many code blocks as you need.
# Warning: run these once when populating the database. No need to run them again just for prediction!
# 20 min
os.makedirs("tmp_data", exist_ok=True)
downloaded_pathnames = fynesse.access.download_price_data()
print(f"Downloaded files: {downloaded_pathnames}")
fynesse.access.create_pp_table(pp_database_conn)
# 15m
fynesse.access.upload_files_to_table(pp_database_conn, downloaded_pathnames, 'pp_data')
# 11m
fynesse.access.setup_pp_table(pp_database_conn)

In [ ]:
# Warning: run these once when populating the database. No need to run them again just for prediction!
fynesse.access.create_postcode_table(pp_database_conn)
# 10s
postcode_filename = fynesse.access.download_postcode_data()
# 1m
fynesse.access.upload_files_to_table(pp_database_conn, [postcode_filename], 'postcode_data')
# 1m
fynesse.access.setup_postcode_table(pp_database_conn)

In [ ]:
# Warning: run these once when populating the database. No need to run them again just for prediction!
fynesse.access.create_prices_coordinates_table(pp_database_conn)
# 20m
joined_table_pathnames = fynesse.access.join_all_tables(downloaded_pathnames, postcode_filename, overwrite=False)
# 10m
fynesse.access.upload_files_to_table(pp_database_conn, joined_table_pathnames, 'prices_coordinates_data', ignore_first_row=True)
# 9m
fynesse.access.setup_prices_coordinates_table(pp_database_conn)

## Question 2. Accessing OpenStreetMap and Assessing the Available Features

In question 3 you will be given the task of constructing a prediction system for property price levels at a given location. We expect that knowledge of the local region around the property should be helpful in making those price predictions. To evaluate this we will now look at [OpenStreetMap](https://www.openstreetmap.org) as a data source.

The tasks below will guide you in accessing and assessing the OpenStreetMap data. The code you write will eventually be assimilated in your python module, but documentation of what you've included and why should remain in the notebook below.

Accessing OpenStreetMap through its API can be done using the python library `osmx`. Using what you have learned about the `osmx` interface in the lectures, write general code for downloading points of interest and other relevant information that you believe may be useful for predicting house prices. Remembering the perspectives we've taken on *data science as debugging*, the remarks we've made when discussing *the data crisis* of the importance of reusability in data analysis, and the techniques we've explored in the labsessions for visualising features and exploring their correlation use the notebook to document your assessment of the OpenStreetMap data as a potential source of data.

The knowledge you need to do a first pass through this question will have been taught by end of lab session three (16th November 2021). You will likely want to review your answer as part of *refactoring* your code and analysis pipeline shortly before hand in.

You should write reusable code that allows you to explore the characteristics of different points of interest. Looking ahead to question 3 you'll want to incorporate these points of interest in your prediction code.

*5 marks*


In [ ]:
# Use this cell and cells below for summarising your analysis and documenting your decision making.

## Model decisions

There are a total of X features, they are:
- latitude

Reasonings for the features:
- aaa

The model is a GLM, with a link of $f(x) = e^x$

Reasonings for the model:
- aaa

## Question 3. Addressing a Property Price Prediction Question

For your final tick, we will be asking you to make house price predictions for a given location, date and property type in the UK. You will provide a function that takes input a latitude and longitude as well as the `property_type` (either type" of property (either `F` - flat, `S` - semidetached, `D` - detached, `T` - terraced or `O` other). Create this function in the `address.py` file, for example in the form,

```
def predict_price(latitude, longitude, date, property_type):
    """Price prediction for UK housing."""
    pass
```

We suggest that you use the following approach when building your prediction.

1. Select a bounding box around the housing location in latitude and longitude.
2. Select a data range around the prediction date.
3. Use the data ecosystem you have build above to build a training set from the relevant time period and location in the UK. Include appropriate features from OSM to improve the prediction.
4. Train a linear model on the data set you have created.
5. Validate the quality of the model.
6. Provide a prediction of the price from the model, warning appropriately if your validation indicates the quality of the model is poor.

Please note that the quality of predictions is not the main focus of the assignment - we expect to see models that output reasonable predictions and have positive R^2's, but you should not spend too much time on increasing the model's accuracy.

The knowledge you need to do a first pass through this question will have been taught by end of lab session four (7th November 2023). You will likely want to review your answer as part of *refactoring* your code shortly before hand in.



In [8]:
from datetime import date
fynesse.assess.predict_price(52.206767, 0.119229, date(2021, 1, 1), 'S', pp_database_conn)

d:\Projects\ads\fynesse\address.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


==== Validation of current model, level 2 ====
No warning issued
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  686
Model:                            GLM   Df Residuals:                      671
Model Family:                Gaussian   Df Model:                           14
Link Function:               Identity   Scale:                      1.7991e+12
Method:                          IRLS   Log-Likelihood:                -10645.
Date:                Fri, 24 Nov 2023   Deviance:                   1.2072e+15
Time:                        20:19:58   Pearson chi2:                 1.21e+15
No. Iterations:                     3   Pseudo R-squ. (CS):             0.2064
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------

873685.6843451262

## Large Language Models

If you used LLMs to generate or fix code in this assignment (recommended), briefly summarise the process and prompts you used. What do you think of the integration of LLMs in the data science pipeline?

```GIVE YOUR WRITTEN ANSWER HERE```

- Some other questions are answered in [this reddit forum](https://www.reddit.com/r/CST_ADS/) or [this doc](https://docs.google.com/document/d/1GfDROyUW8HVs2eyxmJzKrYGRdVyUiVXzPcDfwOO8wX0/edit?usp=sharing). Feel free to also ask about anything that comes up.